# <center>TP2</center>
# <center>Grupo 11</center>

#Integrantes:
- Blas Chuc - 110253
- Franco Rodriguez - 108799
- Helen Chen - 110195
- Tomas Caporaletti - 108598

##Importar librerías

In [ ]:
#Básico
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

#Descarga de modelos
from joblib import dump

##Entrenamiento
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


##Metricas
from sklearn import metrics

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

True

##Analisis de los datos

Descarga de train.csv y test.csv

In [ ]:
!gdown --id 1eVXrJ4w7Gn6FZI7gl-GgucHTmw9105th

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1eVXrJ4w7Gn6FZI7gl-GgucHTmw9105th
To: /content/train.csv
100% 72.2M/72.2M [00:01<00:00, 48.3MB/s]


In [ ]:
!gdown --id 1KNzJ7RtqGMAQEEWZqrQ8G9j8yim1GNi1

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1KNzJ7RtqGMAQEEWZqrQ8G9j8yim1GNi1
To: /content/test.csv
100% 11.1M/11.1M [00:00<00:00, 34.1MB/s]


In [ ]:
df_test = pd.read_csv('test.csv')

In [ ]:
df_train = pd.read_csv('train.csv')

In [ ]:
display(df_train.head())

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo


In [ ]:
df_test.head()

,ID,review_es
0,60000,La mayor virtud de esta película es su existen...
1,60001,"No soy un experto cinéfilo, pero pocas veces m..."
2,60002,Si no eres un incondicional del humor estilo T...
3,60003,"No sé qué está pasando, si la gente se deja ll..."
4,60004,"Pero cuando amanece,y me quedo solo,siento en ..."


##Preprocesamiento

In [ ]:
# columnas
COLUMNA_ID = 'ID'
COLUMNA_REVIEW = 'review_es'
COLUMNA_SENTIMIENTO = 'sentimiento'

# Preparar datos de entrenamiento
X_raw = df_train[COLUMNA_REVIEW]
# Mapear etiquetas a números (1=positivo, 0=negativo)
y = df_train[COLUMNA_SENTIMIENTO].map({'positivo': 1, 'negativo': 0})

# Preparar datos de test
X_test_final_raw = df_test[COLUMNA_REVIEW]
test_ids_final = df_test[COLUMNA_ID]

print(f"Datos de entrenamiento: {len(df_train)} filas")
print(f"Datos de test: {len(df_test)} filas\n")

Datos de entrenamiento: 50000 filas
Datos de test: 8599 filas



In [ ]:
print("CELDA 3: Definiendo la función de preprocesamiento de texto (NLTK)...")

nltk.download('stopwords')

# Stemmer en español

stemmer = SnowballStemmer('spanish')

# Salvamos las negaciones
stop_words_es = set(stopwords.words('spanish'))
palabras_a_mantener = {'no', 'ni', 'sin', 'pero', 'nada', 'poco', 'muy', 'nunca', 'jamás'}
stop_words_es = stop_words_es - palabras_a_mantener

def preprocess_text_stem(text):
    # Convertir a minúsculas
    text = str(text).lower()

    # Eliminar HTML
    text = re.sub(r'<[^>]+>', ' ', text)

    # Eliminar caracteres especiales pero dejar letras y tildes
    text = re.sub(r'[^a-záéíóúñ\s]', '', text)

    words = text.split()

    # Aplicar Stemming y filtrar stopwords
    cleaned_words = [
        stemmer.stem(word)
        for word in words
        if word not in stop_words_es and len(word) > 1
    ]

    return ' '.join(cleaned_words)


CELDA 3: Definiendo la función de preprocesamiento de texto (NLTK)...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Entrenamiento

In [ ]:
X_processed = X_raw.apply(preprocess_text_stem)
X_test_final_processed = X_test_final_raw.apply(preprocess_text_stem)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_processed,
    y,
    test_size=0.20,  # 20% para validación
    random_state=42, # Para reproducibilidad
    stratify=y       # Mantiene el balance de clases [2, 4, 5, 6, 7, 8, 9, 10]
)

In [ ]:
METRICA_SCORING = 'f1_macro'

print(f"Datos para entrenamiento (X_train): {len(X_train)} filas")
print(f"Datos para validación (X_val):   {len(X_val)} filas")
print(f"Métrica de optimización: {METRICA_SCORING}\n")

Datos para entrenamiento (X_train): 40000 filas
Datos para validación (X_val):   10000 filas
Métrica de optimización: f1_macro



In [ ]:
# NAIVE BAYES (con GridSearchCV)

# 1. Pipeline
pipeline_nb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

# 2. Grid
param_grid_nb = {
    # N-gramas: Probamos hasta trigramas para capturar por ej "no me gusto"
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],

    # Filtros de vocabulario
    'tfidf__min_df': [2],         # Ignorar palabras únicas (ruido/typos)
    'tfidf__max_df': [0.5, 0.9],  # Ignorar palabras que aparecen en más del 50% o 90% de docs


    'tfidf__binary': [True, False],  # ¿Importa la frecuencia o solo la presencia?
    'tfidf__sublinear_tf': [True],   # Suaviza picos de frecuencia (bueno para textos largos)

    # Hiperparámetro
    'clf__alpha': [0.1, 0.5, 1.0]
}

# 3. Configuración de la búsqueda
grid_nb = GridSearchCV(
    pipeline_nb,
    param_grid_nb,
    cv=5,
    scoring=METRICA_SCORING,
    n_jobs=-1,
    verbose=1
)

# 4. TRAIN
print("Entrenando GridSearchCV para Naive Bayes...")
grid_nb.fit(X_train, y_train)

# 5. Resultados
print(f"\nMejor puntuación (CV {METRICA_SCORING}): {grid_nb.best_score_:.4f}")
print(f"Mejores parámetros (NB): {grid_nb.best_params_}")


y_pred_nb = grid_nb.predict(X_val)

print(f"F1-Score (Validación): {f1_score(y_val, y_pred_nb, average='macro'):.4f}\n")

Entrenando GridSearchCV para Naive Bayes...
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Mejor puntuación (CV f1_macro): 0.8768
Mejores parámetros (NB): {'clf__alpha': 0.1, 'tfidf__binary': False, 'tfidf__max_df': 0.9, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 3), 'tfidf__sublinear_tf': True}
F1-Score (Validación): 0.8810



In [ ]:
import pickle

nombre_archivo = 'naive_bayes_optimizado.pkl'

with open(nombre_archivo, 'wb') as archivo:
    pickle.dump(grid_nb.best_estimator_, archivo)

print(f"Se guardo el modelo en '{nombre_archivo}'")

Se guardo el modelo en 'modelo_naive_bayes_optimizado.pkl'


#### Métricas

TRAIN

In [ ]:
y_pred_train = grid_nb.predict(X_train)

accuracy_train = accuracy_score(y_train, y_pred_train)
precision_train = precision_score(y_train, y_pred_train, average='binary')
recall_train = recall_score(y_train, y_pred_train, average='binary')
f1_train = f1_score(y_train, y_pred_train, average='binary')


resultados_train_dict = {
    'Métrica': ['F1-Score', 'Precision', 'Recall', 'Accuracy'],
    'Valor (Train)': [f1_train, precision_train, recall_train, accuracy_train]
}

df_resultados_train = pd.DataFrame(resultados_train_dict)

print("\n--- 📊 Cuadro de Resultados (Conjunto de TRAIN) ---")

display(df_resultados_train.style.format({'Valor (Train)': '{:.4f}'}).background_gradient(cmap='Greens'))


print("\n--- Reporte de Clasificación Detallado (TRAIN) ---")
print(classification_report(y_train, y_pred_train))


--- 📊 Cuadro de Resultados (Conjunto de TRAIN) ---


,Métrica,Valor (Train)
0,F1-Score,0.9964
1,Precision,0.9968
2,Recall,0.9961
3,Accuracy,0.9964



--- Reporte de Clasificación Detallado (TRAIN) ---
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20000
           1       1.00      1.00      1.00     20000

    accuracy                           1.00     40000
   macro avg       1.00      1.00      1.00     40000
weighted avg       1.00      1.00      1.00     40000



TEST

In [ ]:
accuracy = accuracy_score(y_val, y_pred_nb)
precision = precision_score(y_val, y_pred_nb, average='binary')
recall = recall_score(y_val, y_pred_nb, average='binary')
f1 = f1_score(y_val, y_pred_nb, average='binary')

resultados_dict = {
    'Métrica': ['F1-Score', 'Precision', 'Recall', 'Accuracy'],
    'Valor (Validación)': [f1, precision, recall, accuracy]
}

df_resultados = pd.DataFrame(resultados_dict)

print("\n--- 📊 Cuadro de Resultados (Conjunto de Validación) ---")
display(df_resultados.style.format({'Valor (Validación)': '{:.4f}'}).background_gradient(cmap='Blues'))

# Reporte detallado por clase
print("\n--- Reporte de Clasificación Detallado ---")
print(classification_report(y_val, y_pred_nb))


--- 📊 Cuadro de Resultados (Conjunto de Validación) ---


,Métrica,Valor (Validación)
0,F1-Score,0.8816
1,Precision,0.8772
2,Recall,0.8860
3,Accuracy,0.8810



--- Reporte de Clasificación Detallado ---
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      5000
           1       0.88      0.89      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



## Predicción

In [ ]:
test_predictions_numeric = grid_nb.predict(X_test_final_processed)

In [ ]:
test_predictions_labels = pd.Series(test_predictions_numeric).map({1: 'positivo', 0: 'negativo'})

In [ ]:
print(test_predictions_labels)

0       negativo
1       negativo
2       negativo
3       positivo
4       negativo
          ...   
8594    positivo
8595    negativo
8596    negativo
8597    negativo
8598    negativo
Length: 8599, dtype: object


## Resultado

In [ ]:
submission_df = pd.DataFrame({
    COLUMNA_ID: test_ids_final,
    COLUMNA_SENTIMIENTO: test_predictions_labels
})
submission_filename = 'grid_nb_stem.csv'
submission_df.to_csv(submission_filename, index=False)

In [ ]:
print(submission_df)